In [ ]:
{'xgboost':dict(
    n_estimators=[2000,5000,10000],
    subsample=[0.8,0.9],
    max_depth=[1,2,3,6], 
)}

## Reducing with XGBoost

In [97]:
f.set_estimator('xgboost')
f.ingest_grid(grids['xgboost'])
f.tune()
f.auto_forecast(**model_params)
f.save_feature_importance()

In [99]:
pfi_fi_xgboost = f.export_feature_importance('xgboost').reset_index()

reduce_vars(
    f,
    pfi_fi_xgboost,
    'xgboost_pfi_reduced',
    patience=5,
    monitor="TestSetRMSE",
)

beginning TestSetRMSE metric value: 26.90168718897764
dropping cycle132sin
dropping AR93
dropping AR85
dropping AR104
dropping AR29
final model trained with 179 variables
no improvement made with feature reduction


In [100]:
export_results(f)

,ModelNickname,TestSetRMSE,InSampleRMSE,TestSetR2,InSampleR2,HyperParams,N_Xvars
0,lasso,25.5482,23.7973,0.8639,0.8820,{'alpha': 0.01},184
1,xgboost,26.9017,0.0003,0.8491,1.0000,"{'n_estimators': 10000, 'subsample': 0.9, 'max...",184
2,gbt,28.4293,0.0022,0.8315,1.0000,"{'n_estimators': 10000, 'max_depth': 3, 'max_f...",184
3,gbt_pfi_reduced,71.5184,0.0021,-0.0662,1.0000,"{'n_estimators': 10000, 'max_depth': 3, 'max_f...",179
4,gbt_gini_reduced,78.1102,0.0020,-0.2718,1.0000,"{'n_estimators': 10000, 'max_depth': 3, 'max_f...",179
5,xgboost_pfi_reduced,108.4637,0.0004,-1.4523,1.0000,"{'n_estimators': 10000, 'subsample': 0.9, 'max...",179
6,mlr,1068.2933,23.4756,-236.8941,0.8852,{},184
7,lasso_coef_reduced,1504.4396,23.7974,-470.7937,0.8820,{'alpha': 0.01},179
8,lasso_pfi_reduced,8187.4196,28.3338,-13972.2264,0.8327,{'alpha': 0.01},179
9,mlr_pfi_reduced1,3181539987170.9668,33.6840,-2109975308996670128128.0000,0.7636,{},89


## MLR

### Train model with all variables

In [60]:
f.set_estimator('mlr')
f.manual_forecast(**model_params)
f.save_feature_importance()

### Reduce with Permutation Feature Importance
- Run and rerun model removing one variable at a time until all negatively contributing variables are filtered out

In [61]:
pfi_fi_mlr = f.export_feature_importance('mlr').reset_index()
pfi_fi_mlr.tail()

,feature,weight,std
179,AR4,-17667.6467,3446.1357
180,AR6,-17825.8849,7681.6496
181,AR2,-19292.1347,3782.6414
182,AR9,-39017.1849,11306.2112
183,AR1,-145745.5705,33726.3446


In [70]:
reduce_vars(
    f,
    pfi_fi_mlr,
    'mlr_pfi_reduced',
    monitor="TestSetRMSE",
    patience=5,
)

beginning TestSetRMSE metric value: 1068.2932679835908
dropping AR1
dropping AR2
dropping AR3
dropping AR4
dropping AR6
final model trained with 179 variables


In [71]:
export_results(f)

,ModelNickname,TestSetRMSE,InSampleRMSE,TestSetR2,InSampleR2,HyperParams,N_Xvars
0,mlr,1068.2933,23.4756,-236.8941,0.8852,{},184
1,mlr_pfi_reduced1,3181539987170.9668,33.6840,-2109975308996670128128.0000,0.7636,{},89
2,mlr_pfi_reduced2,12982958698836.6152,28.2464,-35135808562227792838656.0000,0.8337,{},177
3,mlr_pfi_reduced,29133619050820.1172,27.5971,-176925925212836272799744.0000,0.8413,{},179
